# Notebook principal de Vincent, projet Statapp

### chargement des libraries

In [1]:
!pip install pyarrow==15.0
!pip install s3fs==2024.2.0
!pip install --quiet transformers
!pip install --quiet datasets s3fs
!pip install --quiet transformers[torch]
!pip install --quiet accelerate -U
!pip install --quiet matplotlib
!pip install --quiet seaborn
!pip install --quiet -U scikit-learn
!pip install --quiet nltk
!pip install --quiet wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 41.3 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 15.0.2
    Uninstalling pyarrow-15.0.2:
      Successfully uninstalled pyarrow-15.0.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.9/170.9 kB 4.0 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.3.1
    Uninstalling fsspec-2024.3.1:
      Successfully uninstalled fsspec-2024.3.1
  Attempting uninstall: s3fs
    Found existing installation: s3fs 2024.3.1
    Uninstalling s3fs-2024.3.1:
      Successfully uninstalled s3fs-2024.3.1


Si vous n'utilisez pas le service du ssp cloud:

In [2]:
#pip install torch

### Importation des libraries

In [2]:
from collections import defaultdict
from tqdm import tqdm
import requests
import torch

from transformers import pipeline
from transformers import GPT2Tokenizer, AutoTokenizer
from datasets import load_dataset, Dataset, DatasetDict
from transformers import GPT2LMHeadModel, AutoConfig
from transformers import DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
from transformers import EarlyStoppingCallback

import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import torch.nn.functional as F

import wandb
from datasets import load_metric

import shutil

# Modules maison
import script_training
import selection_corpus

# Pipeline d'entraînement

## récupération dans mes fichiers du ssp cloud ensae des données et préprocessing

In [3]:
import os
import s3fs
import json
# Create filesystem object
S3_ENDPOINT_URL = "https://" + os.environ["AWS_S3_ENDPOINT"]
fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': S3_ENDPOINT_URL})
LOGIN = "vincentg" # Le login Datalab, change d'une personne à l'autre.

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"       # plus besoin de toucher à ça ensuite
os.environ["WAND_PROJECT"] = "exprmt"
os.environ["WANDB_LOG_MODEL"] = "true"

In [4]:
tokenizer = AutoTokenizer.from_pretrained("gpt2-medium")
accuracy_metric = load_metric("accuracy")

/opt/mamba/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/tmp/ipykernel_287/2091263331.py:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  accuracy_metric = load_metric("accuracy")
/opt/mamba/lib/python3.12/site-packages/datasets/load.py:759: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [5]:
# Téléchargement des data de test: https://huggingface.co/datasets/cnn_dailymail?row=0

data_test = load_dataset("cnn_dailymail",  '1.0.0')
data_test = data_test["test"]
encoded_test= tokenizer("\n\n".join(data_test["article"][:300]), return_tensors="pt")

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (204288 > 1024). Running this sequence through the model will result in indexing errors


In [6]:
wandb.login() # rentrez le mdp: **************************

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········


wandb: Appending key for api.wandb.ai to your netrc file: /home/onyxia/.netrc


True

In [8]:
#tokenizer, model, trainer, name = script_training.train_phase(train_val_size = [0.95, 0.05],n_embd=64, n_layer=4, n_head=4, batch_size = 16, epochs=1 )

In [74]:
%%capture captured_output
script_training.main(file='Vendi_1200_100_238.json',
               n_embd=384,
               n_layer=6, 
               n_head=6, 
               train_val_size=[0.90, 0.1], 
               learning_rate=0.002, 
               batch_size=16, 
               epochs=2, 
               warmup_steps=60)

In [ ]:
%%capture captured_output
script_training.main(file='Random_1200_233.json',
          n_embd=384,
          n_layer=6, 
          n_head=6, 
          train_val_size=[0.90, 0.1], 
          learning_rate=0.002, 
          batch_size=16, 
          epochs=2, 
          warmup_steps=60)

In [98]:
print(captured_output.stdout)

## Fonction pour importer

In [76]:
def download_directory(s3_path, local_path, s3=fs):
    # Liste tous les objets (fichiers et sous-répertoires) dans le répertoire S3
    objects = s3.ls(s3_path, detail=True)

    # Crée le répertoire local s'il n'existe pas déjà
    os.makedirs(local_path, exist_ok=True)

    for obj in objects:
        # Construit le chemin local pour chaque objet
        local_obj_path = os.path.join(local_path, os.path.basename(obj['Key']))

        if obj['StorageClass'] == 'DIRECTORY':
            # Si l'objet est un sous-répertoire, appelle la fonction de manière récursive
            download_directory(obj['Key'], local_obj_path)
        else:
            # Si l'objet est un fichier, télécharge-le localement
            s3.get(obj['Key'], local_obj_path)

## TEST sur GPT2 pretrained

In [48]:
model = GPT2LMHeadModel.from_pretrained("gpt2").to("cuda")

Embedding(50257, 768)

In [ ]:
%%capture captured_output
script_training.main(file='tokenized_256_top100.json', model_=model, reprise="GPT2_pretrained_finetune_0.0006_0.5_top_100", learning_rate=0.0006, train_val_size=[0.5, 0.02])
print(captured_output.stdout) 

## Selection des corpus

In [62]:
with fs.open(LOGIN+"/StatApp/top_10_sur_10000.json", 'r') as file:
    data = Dataset.from_list(json.load(file))

In [70]:
%%time
X = selection_corpus.select_least_similar3(data["text"], 
                      size_selected = 1200,         
                      num_subcorpus =  1,    
                      tol = 10**(-2), 
                      affiche_entrop = False ) 

100%|██████████| 1/1 [00:52<00:00, 52.99s/it]

CPU times: user 21min 15s, sys: 1.82 s, total: 21min 17s
Wall time: 54.1 s


In [71]:
X = X.map(lambda x :{'len': len(x["text"])})

Map:   0%|          | 0/1200 [00:00<?, ? examples/s]

In [72]:
sum(X['len'])*2/1000**2

233.438234

In [68]:
# Choisir un Tokenizer
tokenizer = AutoTokenizer.from_pretrained("gpt2-medium")
#tokenizer = GPT2Tokenizer.from_pretrained('gpt2-medium')

def tokenize(element, tokenizer=None, context_length=256):
    outputs = tokenizer(
        element["text"],
        truncation=True,
        max_length= context_length,
        return_overflowing_tokens=True,
        return_length=True,
    )
    input_batch = []
    for length, input_ids in zip(outputs["length"], outputs["input_ids"]):
        if length == context_length:
            input_batch.append(input_ids)
    return {"input_ids": input_batch}




def tokenized_to_cloud(data = None, context_length=256, user = "vincentg", file_name=None):    
    tokenized_datasets = data.map(
        lambda element: tokenize(element, tokenizer=tokenizer),
        batched=True,remove_columns=data.column_names)
    FILE_CHUNK_SHUFFLED_OUT_S3 = user + "/StatApp/" + file_name + ".json"
    with fs.open(FILE_CHUNK_SHUFFLED_OUT_S3, 'w') as f:
    #json.dump(tokenized_datasets, f)
    #tokenized_datasets.to_json(f)
        json.dump(tokenized_datasets['input_ids'],f)
    print("check")

/opt/mamba/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [73]:
tokenized_to_cloud(X, file_name="Random_1200_233")

Map:   0%|          | 0/1200 [00:00<?, ? examples/s]

check


## Tests divers

In [74]:
from transformers import GPT2Tokenizer, AutoModelForCausalLM
import numpy as np

#tokenizer_test2 = GPT2Tokenizer.from_pretrained("gpt2")
#model_test2 = AutoModelForCausalLM.from_pretrained("openai-community/gpt2")
#tokenizer_test2.pad_token_id = tokenizer_test2.eos_token_id


prompt3 = 'Legolas and Gimli advanced on the orcs, raising their weapons with a harrowing war cry.'
inputs = tokenizer.encode(prompt3, return_tensors="pt").to(DEVICE)


In [75]:

# set top_k = 50 and set top_p = 0.95 and num_return_sequences = 3
sample_outputs = model.generate(
    inputs,
    max_new_tokens=100,
    do_sample=True,
    top_k=50,
    top_p=0.95,
    num_return_sequences=3,
    pad_token_id=tokenizer.eos_token_id
)

print("Output:\n" + 100 * '-')
for i, sample_output in enumerate(sample_outputs):
  print("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))

Output:
----------------------------------------------------------------------------------------------------
0: Legolas and Gimli advanced on the orcs, raising their weapons with a harrowing war cry.
: by The the a the and with the in and ( the || is" was and of the
 to, at " || ( a with the
 16,E,, km) that. ||. || was on) and for is= of b —6 the) the  || align || with by in:,. for

 of-E the"
d
. for aid's and ||E were| "  The
 and —.. of
1: Legolas and Gimli advanced on the orcs, raising their weapons with a harrowing war cry. to
's to on was for's- the || and that a ( the in || a of the and || to andg of to and|
 -.  ||| on — ||. the and || of right6- align for that andg'sd) the wasg to
 (
. ||. to-'s of the= | fromd
 - " on to by of with in the. in || tod || The of on || tog
2: Legolas and Gimli advanced on the orcs, raising their weapons with a harrowing war cry. as'sg ('s, — km"- by|), || by was, (. at align to the of the that.,=6 of that ||, as. that with'sE and a to9 in ", || 

In [76]:
beam_outputs = model.generate(
    inputs,
    max_new_tokens=100,
    num_beams=5,
    no_repeat_ngram_size=2,
    num_return_sequences=5,
    early_stopping=True,
    pad_token_id=tokenizer.eos_token_id
)

# now we have 3 output sequences
print("Output:\n" + 100 * '-')
for i, beam_output in enumerate(beam_outputs):
  print("{}: {}".format(i, tokenizer_test2.decode(beam_output, skip_special_tokens=True)))

Output:
----------------------------------------------------------------------------------------------------
0: Legolas and Gimli advanced on the orcs, raising their weapons with a harrowing war cry.
 the the

, the || ||.. the. ||
|
 || the, ||,. and the-
.,
- || a
 a the and ||  ||-. a,, and
 and.- the in the of the  the ( the a || (
 in. in || and- and,--, a. to
 to the to || in
 of
. of || to. (, ( || — || align

1: Legolas and Gimli advanced on the orcs, raising their weapons with a harrowing war cry.
 the the

, the || ||.. the. ||
|
 || the, ||,. and the-
.,
- || a
 a the and ||  ||-. a,, and
 and.- the in the of the  the ( the a || (
 in. in || and- and,--, a. to
 to the to || in
 of
. of || to. (, ( || —, in,
2: Legolas and Gimli advanced on the orcs, raising their weapons with a harrowing war cry.
 the the

, the || ||.. the. ||
|
 || the, ||,. and the-
.,
- || a
 a the and ||  ||-. a,, and
 and.- the in the of the  the ( the a || (
 in. in || and- and,--, a. to
 to the to ||